<a href="https://colab.research.google.com/github/Luensmann/Bachelorarbeit/blob/main/Trainer/Variome_Trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate
!pip install wandb
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset, Features, Sequence, Value, ClassLabel
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, EarlyStoppingCallback, IntervalStrategy, Trainer, DataCollatorForTokenClassification
import evaluate
import json
import wandb
import os
import numpy as np


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# labels

labels = ['O', 'B-disease', 'I-disease', 'B-body-part', 'I-body-part', 'B-mutation', 'I-mutation', 'B-Physiology', 'I-Physiology', 'B-cohort-patient', 'I-cohort-patient', 'B-size', 'I-size', 'B-gender', 'I-gender', 'B-age', 'I-age', 'B-Concepts_Ideas', 'I-Concepts_Ideas', 'B-Disorder', 'I-Disorder', 'B-gene', 'I-gene', 'B-Phenomena', 'I-Phenomena', 'B-ethnicity', 'I-ethnicity']


In [ ]:
# Load Dataset
dataset = load_dataset("Brizape/Variome_tokenized_split_0404_dev")
dataset = dataset.cast_column("ner_tags", Sequence(feature=ClassLabel(names=labels)))
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3048 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/991 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/762 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Brizape___parquet/Brizape--Variome_tokenized_split_0404_dev-b48007fb3d12079a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Casting the dataset:   0%|          | 0/3048 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/991 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/762 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'texts', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3048
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'texts', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 991
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'texts', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 762
    })
})

In [ ]:
tokenized_dataset = dataset.remove_columns(["id", "tokens", "ner_tags"])
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['texts', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3048
    })
    test: Dataset({
        features: ['texts', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 991
    })
    validation: Dataset({
        features: ['texts', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 762
    })
})

In [ ]:
dataset['train'].features

{'id': Value(dtype='string', id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-disease', 'I-disease', 'B-body-part', 'I-body-part', 'B-mutation', 'I-mutation', 'B-Physiology', 'I-Physiology', 'B-cohort-patient', 'I-cohort-patient', 'B-size', 'I-size', 'B-gender', 'I-gender', 'B-age', 'I-age', 'B-Concepts_Ideas', 'I-Concepts_Ideas', 'B-Disorder', 'I-Disorder', 'B-gene', 'I-gene', 'B-Phenomena', 'I-Phenomena', 'B-ethnicity', 'I-ethnicity'], id=None), length=-1, id=None),
 'texts': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

In [ ]:
# Tokenizer
tokenizer_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, trunction=True , model_max_len=512)

In [ ]:
# padding
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
# metric
seqeval = evaluate.load("seqeval")

In [ ]:
label_list = dataset["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-disease',
 'I-disease',
 'B-body-part',
 'I-body-part',
 'B-mutation',
 'I-mutation',
 'B-Physiology',
 'I-Physiology',
 'B-cohort-patient',
 'I-cohort-patient',
 'B-size',
 'I-size',
 'B-gender',
 'I-gender',
 'B-age',
 'I-age',
 'B-Concepts_Ideas',
 'I-Concepts_Ideas',
 'B-Disorder',
 'I-Disorder',
 'B-gene',
 'I-gene',
 'B-Phenomena',
 'I-Phenomena',
 'B-ethnicity',
 'I-ethnicity']

In [ ]:
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

import numpy as np

labels = labels


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels, mode='strict')
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
id2label = {0: 'O',
             1: 'B-disease',
             2: 'I-disease',
             3: 'B-body-part',
             4: 'I-body-part',
             5: 'B-mutation',
             6: 'I-mutation',
             7: 'B-Physiology',
             8: 'I-Physiology',
             9: 'B-cohort-patient',
             10: 'I-cohort-patient',
             11: 'B-size',
             12: 'I-size',
             13: 'B-gender',
             14: 'I-gender',
             15: 'B-age',
             16: 'I-age',
             17: 'B-Concepts_Ideas',
             18: 'I-Concepts_Ideas',
             19: 'B-Disorder',
             20: 'I-Disorder',
             21: 'B-gene',
             22: 'I-gene',
             23: 'B-Phenomena',
             24: 'I-Phenomena',
             25: 'B-ethnicity',
             26: 'I-ethnicity'}
label2id = {'B-Concepts_Ideas': 17,
             'B-Disorder': 19,
             'B-Phenomena': 23,
             'B-Physiology': 7,
             'B-age': 15,
             'B-body-part': 3,
             'B-cohort-patient': 9,
             'B-disease': 1,
             'B-ethnicity': 25,
             'B-gender': 13,
             'B-gene': 21,
             'B-mutation': 5,
             'B-size': 11,
             'I-Concepts_Ideas': 18,
             'I-Disorder': 20,
             'I-Phenomena': 24,
             'I-Physiology': 8,
             'I-age': 16,
             'I-body-part': 4,
             'I-cohort-patient': 10,
             'I-disease': 2,
             'I-ethnicity': 26,
             'I-gender': 14,
             'I-gene': 22,
             'I-mutation': 6,
             'I-size': 12,
             'O': 0}

In [ ]:
# Training several model with different learning rates
ln_rate = [2e-5, 5e-5, 1e-4]

for learnrate in ln_rate:

  runname = "Variome_" + str(learnrate) + "_0404_ES6_strict_tok"

  model = AutoModelForTokenClassification.from_pretrained(
      "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext", num_labels=27, id2label=id2label, label2id=label2id
  )

  training_args = TrainingArguments(
      report_to = 'wandb',                     # enable logging to W&B
      output_dir = runname,    # output directory/ name for huggingface hub
      learning_rate=learnrate,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      weight_decay=0.01,
      evaluation_strategy = 'steps',          # check evaluation metrics at each epoch
      max_steps = 2000,
      logging_steps = 25,                    # we will log every 25 steps
      eval_steps = 25,                      # we will perform evaluation every 25 steps
      save_steps = 25,
      load_best_model_at_end=True,
      metric_for_best_model = 'eval_loss',
      greater_is_better = False,
      push_to_hub=True,
      run_name = runname             # name of the W&B run
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=tokenized_dataset["train"],
      eval_dataset=tokenized_dataset["validation"],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
      callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
  )

  trainer.train()
  wandb.finish()

  trainer.save_model()

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForToken

Download file pytorch_model.bin:   0%|          | 7.40k/416M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.50k/3.50k [00:00<?, ?B/s]

Clean file training_args.bin:  29%|##8       | 1.00k/3.50k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/416M [00:00<?, ?B/s]

...

eval/accuracy,▁▁▁▁▁▁▂▂▂▂▃▄▄▅▅▆▆▅▇▇████
eval/f1,▁▁▁▁▁▁▂▂▃▃▄▅▅▅▅▆▆▅▆▇▇█▇█
eval/loss,████▇▅▄▄▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁
eval/precision,▁▁▁▁▁▁▃▃▄▄▄▄▅▅▆▇▆▇▆▇████
eval/recall,▁▁▁▁▁▁▂▂▃▂▄▅▅▅▅▆▆▅▆▇▇▇▇█
eval/runtime,▂▂▇▂▁▁▁▂▂▂▁▂▂▃▂▂▂█▂▂▁▃▁▃
eval/samples_per_second,▇▇▂▇███▇▇▇█▇▇▆▇▇▇▁▇▇█▆█▆
eval/steps_per_second,▇▇▂▇███▇▇▇█▇▇▆▇▇▇▁▇▇█▆█▆
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,██▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▁▁


Upload file pytorch_model.bin:   0%|          | 1.00/416M [00:00<?, ?B/s]

To https://huggingface.co/Brizape/Variome_2e-05_0404_ES6_strict_tok
   e9dc55f..401b2ec  main -> main

   e9dc55f..401b2ec  main -> main

To https://huggingface.co/Brizape/Variome_2e-05_0404_ES6_strict_tok
   401b2ec..deac22d  main -> main

   401b2ec..deac22d  main -> main

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining mo

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
25,0.571800,0.187061,0.000000,0.000000,0.000000,0.975914
50,0.182500,0.182406,0.000000,0.000000,0.000000,0.975914
75,0.192700,0.184642,0.000000,0.000000,0.000000,0.975914
100,0.180700,0.181107,0.000000,0.000000,0.000000,0.975914
125,0.169500,0.186082,0.000000,0.000000,0.000000,0.975914
150,0.178200,0.180726,0.000000,0.000000,0.000000,0.975914
175,0.174000,0.179813,0.000000,0.000000,0.000000,0.975914
200,0.154700,0.170559,0.000000,0.000000,0.000000,0.975914
225,0.153900,0.140182,0.000000,0.000000,0.000000,0.975949
250,0.141500,0.128137,0.333333,0.000960,0.001914,0.975914


/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-d

In [ ]:
wandb.finish()
trainer.save_model()

eval/accuracy,▁▁▇▇███
eval/f1,▁▂▇██▇█
eval/loss,█▃▂▁▁▂▁
eval/precision,▁▆█▇█▇█
eval/recall,▁▁▆█▇▇▇
eval/runtime,▁▃▄▆▇█▆
eval/samples_per_second,█▆▅▃▂▁▃
eval/steps_per_second,█▆▅▃▂▁▃
train/epoch,▁▁▂▂▃▃▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▅▅▆▆▇▇███
train/learning_rate,█▇▆▅▃▂▁


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.
To https://huggingface.co/Brizape/SETH_5e-05_0404_ES6_strict_tok
   b4087f7..42d5266  main -> main

   b4087f7..42d5266  main -> main

